# HoroRF
> Comparing two hyperbolic RF methods

In [6]:
%load_ext autoreload
%autoreload 2

In [6]:
%%bash

# Run HoroRF:at 1000 
cd HoroRF
/home/phil/mambaforge/envs/hdt/bin/python train_hyp_rf.py -h
mv ./logs/output ./logs/output_$(date +%Y%m%d_%H%M%S)_hororf

2023-09-18 09:03:44,089 [hororf.rf_trainer] 1 GPUs available
2023-09-18 09:03:44,089 [hororf.rf_trainer] Using seed 17 on class 2
2023-09-18 09:03:44,094 [hororf.utils] 977 datapoints in dataset 'datasets.polblogs_geomstats'
2023-09-18 09:03:44,094 [hororf.utils] 977 datapoints in test dataset 'datasets.polblogs_geomstats'
2023-09-18 09:03:44,095 [hororf.rf_trainer] 781 train and 196 test samples for fold 0
2023-09-18 09:06:06,657 [hororf.rf_trainer] Hyperbolic tree f1 micro: 0.8878, f1 macro: 0.8870, AUPR: 0.0000. Mean depth of 6.00
2023-09-18 09:06:06,658 [hororf.rf_trainer] 781 train and 196 test samples for fold 1
2023-09-18 09:08:48,420 [hororf.rf_trainer] Hyperbolic tree f1 micro: 0.9337, f1 macro: 0.9332, AUPR: 0.0000. Mean depth of 6.00
2023-09-18 09:08:48,421 [hororf.rf_trainer] 782 train and 195 test samples for fold 2
2023-09-18 09:11:20,500 [hororf.rf_trainer] Hyperbolic tree f1 micro: 0.9385, f1 macro: 0.9384, AUPR: 0.0000. Mean depth of 6.00
2023-09-18 09:11:20,501 [horor

In [48]:
# For using hororf outputs
# vals = [
#     0.8878, 0.9337, 0.9385, 0.8974, 0.9385
# ]

import numpy as np

dim = 16
dataname = "gaussian"
# dataname = "neuroseed"

for suffix in ["hrf", "results_micro", "rf"]:
    vals = np.loadtxt(f"./HoroRF/logs/big_bench/hororf_{dataname}_{dim}/{suffix}.txt", delimiter="\t")
    print(suffix, f"{np.mean(vals) * 100:.2f}", f"{np.std(vals)*100:.2f}")

hrf 99.79 0.25
results_micro 98.66 0.90
rf 99.79 0.25


In [13]:
# For 16-dimensional embeddings, HoroRF had a micro-F1 score of 0.675. Let's try ours:

import numpy as np
import yaml

from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from src.hyperdt.forest import HyperbolicRandomForestClassifier
from src.hyperdt.conversions import convert

# Read params from yml file


def evaluate_hdt():
    params = yaml.safe_load(open("HoroRF/params.yml", "r"))

    # Dataset
    print(f"Using loader from file: {params['dataset_file']}")
    print()  # For tqdm compatibility
    if params["dataset_file"] == "datasets.gaussian":
        from HoroRF.datasets.gaussian import get_training_data, get_testing_data
    elif params["dataset_file"] == "datasets.neuroseed":
        from HoroRF.datasets.neuroseed import get_training_data, get_testing_data
    elif params["dataset_file"] == "datasets.polblogs_geomstats":
        from HoroRF.datasets.polblogs_geomstats import get_training_data, get_testing_data

    # Get data
    X_train, y_train = get_training_data(class_label=params["class_label"], seed=params["seed"])
    X_train = convert(X_train.numpy(), "poincare", "hyperboloid")
    X_test, y_test = get_testing_data(class_label=params["class_label"], seed=params["seed"])
    X_test = convert(X_test.numpy(), "poincare", "hyperboloid")

    # Hyperparams
    args = {
        "n_estimators": params["num_trees"],
        "max_depth": params["max_depth"],
        "min_samples_leaf": params["min_samples_leaf"],
    }

    # 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=params["seed"])
    f1_scores_hrf = []
    f1_scores_rf = []
    for train_index, test_index in kf.split(X_train):
        # Hyperbolic
        hrf = HyperbolicRandomForestClassifier(**args)
        hrf.fit(X_train[train_index], y_train[train_index], use_tqdm=True, seed=params["seed"])
        y_pred = hrf.predict(X_train[test_index])
        f1_scores_hrf.append(f1_score(y_train[test_index], y_pred, average="micro"))

        # Euclidean
        rf = RandomForestClassifier(**args, random_state=params["seed"])
        rf.fit(X_train[train_index], y_train[train_index])
        y_pred = rf.predict(X_train[test_index])
        f1_scores_rf.append(f1_score(y_train[test_index], y_pred, average="micro"))

    return f1_scores_hrf, f1_scores_rf


f1_scores_hrf, f1_scores_rf = evaluate_hdt()
print(f"Hyperbolic: {np.mean(f1_scores_hrf):.3f} +/- {np.std(f1_scores_hrf):.3f}")
print(f"Euclidean: {np.mean(f1_scores_rf):.3f} +/- {np.std(f1_scores_rf):.3f}")

Using loader from file: datasets.neuroseed



100%|██████████| 24/24 [00:00<00:00, 29.08it/s]


Hyperbolic: 0.867 +/- 0.012
Euclidean: 0.885 +/- 0.016


In [12]:
from HoroRF.datasets.gaussian import get_training_data, get_testing_data

get_training_data(class_label=2, seed=0)[0].shape

torch.Size([1000, 2])

In [42]:
from HoroRF.datasets.neuroseed import get_training_data, get_testing_data

get_training_data(class_label=2, seed=0)[1]

tensor([44, 44,  9, 22, 44, 44,  9, 44, 44,  9,  9, 22, 22, 44, 22, 44, 22, 22,
        44, 22,  3, 22, 22, 44, 44, 44,  2,  3, 22, 22, 22,  2, 22,  2, 44, 44,
        22,  9, 44, 44, 22, 44, 22, 22, 22,  9,  9, 44, 44, 22, 44,  3, 44, 22,
         9, 43, 44, 22, 22, 44,  9,  3,  9,  2, 44, 43, 44, 22,  9,  3, 44,  9,
        44, 44, 22,  9,  3,  3, 44,  2, 22,  2,  3, 22, 44,  3,  3,  3, 44, 44,
        44,  2,  2, 22, 22, 22,  3,  2,  9, 22])

In [39]:
labels = adata.var["taxonomy_1"]
labels_counts = labels.value_counts()
keep = labels_counts[labels_counts > 1000].index

labels_filtered = labels[labels.isin(keep)]
labels_filtered.index

Index(['776992', '1050608', '190299', '358030', '239283', '4030157', '35786',
       '174924', '370251', '191389',
       ...
       '268328', '228988', '155616', '158709', '299059', '515774', '311952',
       '568082', '1112813', '562583'],
      dtype='object', length=32863)

In [41]:
import numpy as np

indices = np.random.choice(labels_filtered.index, 125, replace=False)
indices

array(['145236', '4442899', '112801', '269532', '95741', '301910',
       '74869', '1967053', '768535', '691952', '294040', '470879',
       '358439', '95522', '268755', '4448558', '593016', '318205',
       '4298060', '4475224', '1096766', '1108726', '3219862', '193763',
       '2545365', '252198', '516020', '271500', '354401', '241499',
       '4437436', '971971', '344456', '322087', '4371949', '554911',
       '202816', '4444213', '4416974', '548878', '164915', '370295',
       '4445508', '4321043', '4416763', '1087825', '997439', '4256699',
       '3862524', '47181', '174004', '407459', '683241', '4364083',
       '115049', '206331', '343699', '964799', '1667530', '4459355',
       '583472', '4377731', '1105919', '814570', '709691', '145786',
       '332210', '228043', '810672', '199344', '904468', '668257',
       '4322804', '4320437', '4367317', '807112', '280233', '147940',
       '1066654', '4469223', '563671', '2838675', '4468097', '4349553',
       '1074801', '1117187', '9985

In [31]:
# Figure out error with my method:

seed = 15
dim = 4
from HoroRF.datasets.neuroseed import get_training_data, get_testing_data
from src.hyperdt.conversions import convert

X, y = get_training_data(class_label=dim, seed=seed, num_samples=1000)
X_h = convert(X.numpy(), "poincare", "hyperboloid")

In [34]:
from src.hyperdt.forest import HyperbolicRandomForestClassifier

hrf = HyperbolicRandomForestClassifier(n_estimators=24, max_depth=6)

hrf.fit(X_h, y.numpy(), use_tqdm=True, seed=seed)

100%|██████████| 24/24 [00:02<00:00,  9.98it/s]


HyperbolicRandomForestClassifier()

In [35]:
f1_score(y.numpy(), hrf.predict(X_h), average="micro")

0.045

In [8]:
# Troubleshooting poor neuroseed performance

from HoroRF.datasets.neuroseed import get_training_data, get_testing_data
from src.hyperdt.conversions import convert
from src.hyperdt.tree import HyperbolicDecisionTreeClassifier

X, y = get_training_data(class_label=2, seed=0, num_samples=400)
X_h = convert(X.numpy(), "poincare", "hyperboloid")

In [9]:
# So here's somewhere we fail:
# n = 400, seed = 1, dim = 8

import numpy as np

from src.hyperdt.forest import HyperbolicRandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

n = 400
seed = 1
dim = 8

X, y = get_training_data(class_label=dim, seed=seed, num_samples=n)
X_h = convert(X.numpy(), "poincare", "hyperboloid")
y = y.numpy()
# Relabel
_, y = np.unique(y, return_inverse=True)

cv = KFold(n_splits=5, shuffle=True, random_state=seed)

hrf = HyperbolicRandomForestClassifier(n_estimators=1, max_depth=6)

for train_index, test_index in cv.split(X):
    hrf.fit(X[train_index], y[train_index], use_tqdm=True, seed=seed)
    y_pred = hrf.predict(X[test_index])
    print(f1_score(y[test_index], y_pred, average="micro"))

100%|██████████| 1/1 [00:00<00:00, 46.29it/s]


TypeError: sum() received an invalid combination of arguments - got (out=NoneType, axis=int, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: out, axis
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


In [10]:
hrf = HyperbolicRandomForestClassifier(n_estimators=1, max_depth=6)

In [12]:
hrf.fit(X_h, y)

f1_score(y, hrf.predict(X_h), average="micro")

0.925

In [25]:
hrf.classes_

array([ 2,  3,  9, 22, 43, 44])

In [34]:
import numpy as np

reference = hrf.trees[0].classes_

assert np.all(reference == hrf.classes_)
assert np.all(reference == [tree.classes_ for tree in hrf.trees])

In [42]:
# Let's step through each method, I guess

from src.hyperdt.tree import HyperbolicDecisionTreeClassifier

X, y = get_training_data(class_label=4, seed=15, num_samples=1000)
X_h = convert(X.numpy(), "poincare", "hyperboloid")
hdt = HyperbolicRandomForestClassifier(max_depth=6)
hdt.fit(X_h, y)

HyperbolicRandomForestClassifier()

In [44]:
# First up, we see that f1 score is messed up:

from sklearn.metrics import f1_score

y_pred = hdt.predict(X_h)
f1_score(y, hdt.classes_[y_pred], average="micro")

IndexError: index 9 is out of bounds for axis 0 with size 6

In [52]:
hdt.trees[0].classes_

array([ 2,  3,  9, 22, 43, 44])

In [55]:
hdt.classes_

array([ 2,  3,  9, 22, 43, 44])

In [54]:
np.unique(y)

array([ 2,  3,  9, 22, 43, 44])

In [61]:
hdt2 = HyperbolicDecisionTreeClassifier()
hdt2.fit(X_h, y)
f1_score(hdt2.predict(X_h), y, average="micro")

0.86875

In [75]:
hrf2 = HyperbolicRandomForestClassifier()
hrf2.fit(X_h, y)
f1_score(hrf2.predict(X_h), y, average="micro")

0.02375

In [64]:
# So retraining the tree gives us the same issue:
hrf2.trees[0].fit(X_h, y)
f1_score(hrf2.trees[0].predict(X_h), y, average="micro")

0.02375

In [69]:
hrf2.trees[0]._get_probs(y)

array([0.04875, 0.06875, 0.16625, 0.3575 , 0.02625, 0.3325 ])

In [76]:
type(hrf2.trees[0])

src.hyperdt.tree.DecisionTreeClassifier

In [12]:
import numpy as np

seed = 0
dim = 8

from HoroRF.datasets.gaussian import get_training_data, get_testing_data

X, y = get_training_data(class_label=dim, seed=seed, num_samples=1000, convert=False)
X = X.numpy()
y = y.numpy()
# X_h = convert(X.numpy(), "poincare", "hyperboloid")

from src.hyperdt.forest import HyperbolicRandomForestClassifier

hrf = HyperbolicRandomForestClassifier(n_estimators=1, max_depth=6)
hrf.fit(X, y)

ValueError: Points must lie on a hyperboloid: Lorentzian Inner Product does not equal the curvature of 1.

In [13]:
np.sum(X[:, 1:] ** 2, axis=1) - X[:, 0] ** 2

array([-1.        , -1.015625  , -1.        , -1.        , -0.99999952,
       -1.        , -1.        , -1.00000095, -1.        , -1.        ,
       -1.00000381, -1.00000012, -1.        , -1.        , -1.        ,
       -1.00000006, -1.        , -1.00000763, -1.        , -1.        ,
       -1.        , -1.        , -1.        , -0.99999237, -1.        ,
       -0.99999619, -1.        , -1.        , -1.015625  , -1.        ,
       -0.99999976, -0.99999999, -1.        , -1.        , -1.00001526,
       -0.99999997, -1.        , -1.        , -1.00000012, -1.        ,
       -1.00000048, -1.        , -1.        , -1.00000381, -1.        ,
       -1.        , -1.        , -1.        , -1.00000191, -1.        ,
       -1.00000012, -0.99999999, -1.        , -1.        , -0.99999999,
       -1.        , -1.        , -1.        , -1.00000048, -1.        ,
       -1.        , -1.        , -1.        , -1.        , -1.        ,
       -1.        , -1.        , -1.        , -1.        , -1.  